# Hypothesis Testing

In this notebook, we are going to represent the logic of our hypothesis testing for comparing NATS config changes. Everytime that we make a change, we benchmark the before and after change cases. After that, we change the groups names based on ```tmp``` directory names. Then we run this notebook in order to check the compare results.

In [1]:
# notebook variables
GROUP_A="no-syncpub-3/dataset.csv" # change this
GROUP_B="no-syncpub-rep1/dataset.csv" # change this

DEFAULT_DIR = "../runbook/tmp"

# ttest variables
P_VALUE_BOUND = 0.05 # change the pvalue bound
PERMUTATION_VALUE = 25 # you can update permutation value (ususally we set it to half of our dataset capacity)

In [2]:
# import in-use libraries
import pandas as pd
from scipy.stats import ttest_ind

In [3]:
# read csv to create groups datasets
dfA = pd.read_csv(f'{DEFAULT_DIR}/{GROUP_A}')
dfB = pd.read_csv(f'{DEFAULT_DIR}/{GROUP_B}')

dfA.head()

,pub-stats,sub-stats,overall-stats
0,11.66,NaN,NaN
1,11.84,NaN,NaN
2,10.57,NaN,NaN
3,10.14,NaN,NaN
4,11.99,NaN,NaN


In [4]:
# describe datasets
dfA.describe()

,pub-stats,sub-stats,overall-stats
count,50.000000,0.0,0.0
mean,11.949600,NaN,NaN
std,1.230279,NaN,NaN
min,9.250000,NaN,NaN
25%,11.312500,NaN,NaN
50%,11.990000,NaN,NaN
75%,12.662500,NaN,NaN
max,14.140000,NaN,NaN


In [5]:
# using a function for hypothesis testing logic
# which gets two columns data to compare
def hypo_test(groupA, groupB):
    # get mean values
    mean_a = groupA.mean()
    mean_b = groupB.mean()
    
    t_statistic, p_value = ttest_ind(groupA, groupB, equal_var=True, permutations=PERMUTATION_VALUE)
    
    print(f"\tmean of `{GROUP_A}`: {mean_a}")
    print(f"\tmean of `{GROUP_B}`: {mean_b}")
    print(f"\tt-statistic: {t_statistic}")
    print(f"\tp-value: {p_value}")
    
    if p_value < P_VALUE_BOUND:
        print("the difference is statistically significant at 95% confidence level.")
        if mean_a > mean_b:
            print(f"`{GROUP_A}` is better by {100 * float((mean_a-mean_b)/mean_a)}%.")
        else:
            print(f"`{GROUP_B}` is better by {100 * float((mean_b-mean_a)/mean_b)}%.")
    else:
        print("the difference is not statistically significant at 95% confidence level.")

In [6]:
# defining dataset columns
columns = ["pub-stats", "sub-stats", "overall-stats"]

In [7]:
#  hypothesis for comparing two groups
for column in columns:
    print(f"\ntesting `{column}` field:")
    hypo_test(dfA[column], dfB[column])


testing `pub-stats` field:
	mean of `no-syncpub-3/dataset.csv`: 11.9496
	mean of `no-syncpub-rep1/dataset.csv`: 20.551999999999992
	t-statistic: -22.983042981964857
	p-value: 0.0
the difference is statistically significant at 95% confidence level.
`no-syncpub-rep1/dataset.csv` is better by 41.85675360062279%.

testing `sub-stats` field:
	mean of `no-syncpub-3/dataset.csv`: nan
	mean of `no-syncpub-rep1/dataset.csv`: nan
	t-statistic: nan
	p-value: nan
the difference is not statistically significant at 95% confidence level.

testing `overall-stats` field:
	mean of `no-syncpub-3/dataset.csv`: nan
	mean of `no-syncpub-rep1/dataset.csv`: nan
	t-statistic: nan
	p-value: nan
the difference is not statistically significant at 95% confidence level.
